## Dataset Preparation
As data set we use `tiny_shakespeare`, which consists of 40,000 lines of Shakespeare from a variety of Shakespeare's plays. (Featured in Andrej Karpathy's blog post 'The Unreasonable Effectiveness of Recurrent Neural Networks': http://karpathy.github.io/2015/05/21/rnn-effectiveness/)


### Reviewing Data

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget -P ./dataset https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# read it in to inspect it
with open("./dataset/input.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [ ]:
print(f"length of dataset in characters: {len(text)}")

In [ ]:
# let's look at the first 1000 characters
print(text[:500])

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(f"Vocabulary size: {vocab_size}")

### Encoder/Decoder

We want to have a `encoder` function, that maps characters to integers and a `decoder` function resolve those integers back to characters. 

In [ ]:
# creates maps for resolution
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

# encoder: take a string, output a list of integers
encode = lambda s: [stoi[c] for c in s]

# decoder: take a list of integers, output a string
decode = lambda l: "".join([itos[i] for i in l])  # ⚠️⚠️⚠️⚠️⚠️

print(encode("hii there"))
print(decode(encode("hii there")))

#### Bonus: Looking at TikToken

Though we will use a simple character level implementation, lets have a look a existing tokenizers like tiktoken. tiktoken is a fast [BPE](https://en.wikipedia.org/wiki/Byte_pair_encoding) tokeniser for use with OpenAI's models.

In [54]:
import tiktoken

tokenizer = tiktoken.encoding_for_model("gpt-2")
print(tokenizer.encode("hii there"))
print(tokenizer.decode(tokenizer.encode("hii there")))

[71, 4178, 612]
hii there


Let's now encode the entire text dataset and store it into a `torch.Tensor`

In [55]:
import torch  # we use PyTorch: https://pytorch.org

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(
    data[:500]
)  # the 500 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

<div style="text-align:center"><img align="center" src="https://media5.datahacker.rs/2021/11/Picture3.jpg" width="500"/></div><br>
Let's now split up the data into train and validation sets, to avoid overfitting

In [56]:
ratio = 0.9
n = int(ratio * len(data))  # first 90% will be train, rest val
train_data = data[:n]  # ⚠️⚠️⚠️⚠️⚠️
val_data = data[n:]  # ⚠️⚠️⚠️⚠️⚠️

We do not feed the transformer the whole text at once! That would be computational very expensive and prohibitive. Instead we take random like chunks for the dataset an train the transformer on those at a time. The length of those chunks is calles `block_size`

In [57]:
block_size = 8
train_data[: block_size + 1]  # +1 because we need a last target token to have 8 trainings samples. Visualization below

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [58]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


We do not just put a single chunk into the transformer but a `batch` of chunks, stored in a tensor, to keep the GPUs busy. Those chunks will processed at the same time, but they are indepent and don't _talk_ to each other.

In [59]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

We will not export our dataset as well as the meta data

In [60]:
import pickle
import numpy as np

train_ids = np.array(train_data, dtype=np.uint16)
val_ids = np.array(val_data, dtype=np.uint16)

train_ids.tofile('./dataset/train.bin')
val_ids.tofile('./dataset/val.bin')

# save the meta information as well, to help us encode/decode later
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open ('./dataset/meta.pkl', 'wb') as f:
    pickle.dump(meta, f) # ⚠️⚠️⚠️⚠️⚠️  https://docs.python.org/3/library/pickle.html#pickle.dump


/var/folders/q6/jkgby_n11x722gs_023k8sc40000gp/T/ipykernel_22968/1254640617.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  train_ids = np.array(train_data, dtype=np.uint16)
/var/folders/q6/jkgby_n11x722gs_023k8sc40000gp/T/ipykernel_22968/1254640617.py:5: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  val_ids = np.array(val_data, dtype=np.uint16)
